In [1]:
import openfermion as of

# PPP_Model

In [2]:
from openfermion.ops.operators import BosonOperator, FermionOperator
from openfermion.utils.indexing import down_index, up_index
from openfermion.hamiltonians.special_operators import number_operator


def fermi_ppp(z,
            unit_dist,
            x_dimension,
            y_dimension,
            tunneling,
            coulomb,
            chemical_potential=0.,
            magnetic_field=0.,
            periodic=True,
            spinless=False,
            particle_hole_symmetry=False):
   
    if spinless:
        return _spinless_fermi_hubbard_model(x_dimension, y_dimension,
                                             tunneling, coulomb,
                                             chemical_potential, magnetic_field,
                                             periodic, particle_hole_symmetry,z,unit_dist)
    else:
        return _spinful_fermi_hubbard_model(x_dimension, y_dimension, tunneling,
                                            coulomb, chemical_potential,
                                            magnetic_field, periodic,
                                            particle_hole_symmetry,z,unit_dist)


def _spinful_fermi_hubbard_model(x_dimension, y_dimension, tunneling, coulomb,
                                 chemical_potential, magnetic_field, periodic,
                                 particle_hole_symmetry,z,unit_dist):

    # Initialize operator.
    n_sites = x_dimension * y_dimension
    n_spin_orbitals = 2 * n_sites
    hubbard_model = FermionOperator()

    # Loop through sites and add terms.
    for site in range(n_sites):

        # Get indices of right and bottom neighbors
        right_neighbor = _right_neighbor(site, x_dimension, y_dimension,
                                         periodic)
        bottom_neighbor = _bottom_neighbor(site, x_dimension, y_dimension,
                                           periodic)

        # Avoid double-counting edges when one of the dimensions is 2
        # and the system is periodic
        if x_dimension == 2 and periodic and site % 2 == 1:
            right_neighbor = None
        if y_dimension == 2 and periodic and site >= x_dimension:
            bottom_neighbor = None

        # Add hopping terms with neighbors to the right and bottom.
        if right_neighbor is not None:
            hubbard_model += _hopping_term(up_index(site),
                                           up_index(right_neighbor), -tunneling)
            hubbard_model += _hopping_term(down_index(site),
                                           down_index(right_neighbor),
                                           -tunneling)
        if bottom_neighbor is not None:
            hubbard_model += _hopping_term(up_index(site),
                                           up_index(bottom_neighbor),
                                           -tunneling)
            hubbard_model += _hopping_term(down_index(site),
                                           down_index(bottom_neighbor),
                                           -tunneling)

        # Add local pair Coulomb interaction terms.
        hubbard_model += _coulomb_interaction_term(n_spin_orbitals,
                                                   up_index(site),
                                                   down_index(site), coulomb,
                                                   particle_hole_symmetry)

        # Add chemical potential and magnetic field terms.
        hubbard_model += number_operator(n_spin_orbitals, up_index(site),
                                         -chemical_potential - magnetic_field)
        hubbard_model += number_operator(n_spin_orbitals, down_index(site),
                                         -chemical_potential + magnetic_field)
        
        # Add intersite_interaction terms.
        hubbard_model += _intersite_interaction_term(unit_dist, coulomb, n_sites, z)
       
    return hubbard_model


def _spinless_fermi_hubbard_model(x_dimension, y_dimension, tunneling, coulomb,
                                  chemical_potential, magnetic_field, periodic,
                                  particle_hole_symmetry,z,unit_dist):

    # Initialize operator.
    n_sites = x_dimension * y_dimension
    hubbard_model = FermionOperator()

    # Loop through sites and add terms.
    for site in range(n_sites):

        # Get indices of right and bottom neighbors
        right_neighbor = _right_neighbor(site, x_dimension, y_dimension,
                                         periodic)
        bottom_neighbor = _bottom_neighbor(site, x_dimension, y_dimension,
                                           periodic)

        # Avoid double-counting edges when one of the dimensions is 2
        # and the system is periodic
        if x_dimension == 2 and periodic and site % 2 == 1:
            right_neighbor = None
        if y_dimension == 2 and periodic and site >= x_dimension:
            bottom_neighbor = None

        # Add terms that couple with neighbors to the right and bottom.
        if right_neighbor is not None:
            # Add hopping term
            hubbard_model += _hopping_term(site, right_neighbor, -tunneling)
            # Add local Coulomb interaction term
            hubbard_model += _coulomb_interaction_term(n_sites, site,
                                                       right_neighbor, coulomb,
                                                       particle_hole_symmetry)
            # Add intersite_interaction terms.
            hubbard_model += _intersite_interaction_term(unit_dist, coulomb, x_dimension, z)
        if bottom_neighbor is not None:
            # Add hopping term
            hubbard_model += _hopping_term(site, bottom_neighbor, -tunneling)
            # Add local Coulomb interaction term
            hubbard_model += _coulomb_interaction_term(n_sites, site,
                                                       bottom_neighbor, coulomb,
                                                       particle_hole_symmetry)
            # Add intersite_interaction terms.
            hubbard_model += _intersite_interaction_term(unit_dist, coulomb, n_sites, z)

        # Add chemical potential. The magnetic field doesn't contribute.
        hubbard_model += number_operator(n_sites, site, -chemical_potential)

    return hubbard_model


def _hopping_term(i, j, coefficient, bosonic=False):
    op_class = BosonOperator if bosonic else FermionOperator
    hopping_term = op_class(((i, 1), (j, 0)), coefficient)
    hopping_term += op_class(((j, 1), (i, 0)), coefficient.conjugate())
    return hopping_term


def _coulomb_interaction_term(n_sites,
                              i,
                              j,
                              coefficient,
                              particle_hole_symmetry,
                              bosonic=False):
    op_class = BosonOperator if bosonic else FermionOperator
    number_operator_i = number_operator(n_sites, i, parity=2 * bosonic - 1)
    number_operator_j = number_operator(n_sites, j, parity=2 * bosonic - 1)
#     print("Number_Operator")
#     print(number_operator_i)
    if particle_hole_symmetry:
        number_operator_i -= op_class((), 0.5)
        number_operator_j -= op_class((), 0.5)
    return coefficient * number_operator_i * (number_operator_j-1)

def _intersite_interaction_term(unit_dist, U, n_sites, z):
    intersite = None
    n_sites = x_dimension*y_dimension
    for i in range(1, n_sites+1):
        for j in range(1, n_sites+1):
            if i != j:
                vij = _ohno(unit_dist, U, i, j)
                ni, nj = _get_n(z,i, j)
                intersite = 0.5*vij*ni*nj
    return intersite 

# def _intersite_interaction_term(unit_dist, U, n_sites, z):
#     intersite = None
#     n_sites = x_dimension*y_dimension
#     for i in range(1, n_sites+1):
#         x = ()
#         if i == 0: x = x+(i+1,)
#         elif i == n_sites: x = x+(i-1,)
#         else:
#             x = x+(i-1,)
#             x = x+(i+1,)
#         for j in x:
#             if i != j:
#                 vij = _ohno(unit_dist, U, i, j)
#                 ni, nj = _get_n(z,i, j)
#                 intersite = vij*ni*nj
#     return intersite 


def _ohno(unit_dist, U, i, j):
    rij = (j-i)*unit_dist
    vij = 14.397*(((28.794/2*U)**2)+rij**2)**0.5
    return vij

# def _ohno(unit_dist, U, i, j):
#     rij = (j-i)*unit_dist
#     vij = 14.397*(((28.794/2*U)+rij))
#     return vij


def _get_n(z,i, j):
    n_sites = x_dimension*y_dimension
    op_class = FermionOperator
    n_i = number_operator(n_sites, i)
    n_j = number_operator(n_sites, j)
    return (n_i-z, n_j-z)


def _right_neighbor(site, x_dimension, y_dimension, periodic):
    if x_dimension == 1:
        return None
    if (site + 1) % x_dimension == 0:
        if periodic:
            return site + 1 - x_dimension
        else:
            return None
    return site + 1


def _bottom_neighbor(site, x_dimension, y_dimension, periodic):
    if y_dimension == 1:
        return None
    if site + x_dimension + 1 > x_dimension * y_dimension:
        if periodic:
            return site + x_dimension - x_dimension * y_dimension
        else:
            return None
    return site + x_dimension

In [7]:
"""Define the Hamiltonian."""
# Parameters.
nsites = 2
U = 11.26
J = 2.4
z = 1
x_dimension = nsites
y_dimension = 1
unit_dist = 1.33
ppp = fermi_ppp(z,
                  unit_dist,
                  x_dimension,
                  y_dimension,
                  -J,
                  U,
                  chemical_potential=0.,
                  magnetic_field=0.,
                  periodic=True,
                  spinless=False,
                  particle_hole_symmetry=False)
print(ppp)

2333.979383975641 [] +
-11.26 [0^ 0] +
11.26 [0^ 0 1^ 1] +
2.4 [0^ 2] +
-2333.979383975641 [1^ 1] +
2.4 [1^ 3] +
2.4 [2^ 0] +
-2345.2393839756414 [2^ 2] +
2333.979383975641 [2^ 2 1^ 1] +
11.26 [2^ 2 3^ 3] +
2.4 [3^ 1]


In [8]:
from openfermion.linalg import get_sparse_operator, get_ground_state
from openfermion.transforms import jordan_wigner
from openfermion.linalg import eigenspectrum

# Get qubit operator under Jordan-Wigner.
jw_hamiltonian = jordan_wigner(ppp)
# jw_hamiltonian.compress()
# print("jw_hamiltonian")
# print(jw_hamiltonian)
print('')

jw_spectrum = eigenspectrum(jw_hamiltonian)
print("jw_eigenspectrum")
print(jw_spectrum)
print('')

qubit_matrix = get_sparse_operator(jw_hamiltonian)
# print("Qubit Matrix")
# print(qubit_matrix)

exc_energy = jw_spectrum[1]-jw_spectrum[0]
print ("excitation energy",exc_energy)
print('\nEnergy of the model is {} .'.format(
   get_ground_state(qubit_matrix)[0]))


jw_eigenspectrum
[-2.25200000e+01 -1.36600000e+01 -1.22043105e+01 -1.12624679e+01
 -8.86000000e+00 -2.40000000e+00 -2.46788551e-03 -1.09245946e-13
 -8.37489676e-14 -5.47277094e-14  9.39350832e-01  2.40000000e+00
  2.32272185e+03  2.32272434e+03  2.33397938e+03  2.33398185e+03]

excitation energy 8.860000000000117

Energy of the model is -22.52000000000038 .


In [62]:
import math as m
points = [
     [1,2],
     [3,4],
     [5,6]
 ]
    
for i in range(len(points)):
    point_ix, point_iy = points[i][0], points[i][1]
    for j in range(len(points)):
        if i != j:
            point_jx, point_jy = points[j][0], points[j][1]
            distance = m.sqrt((point_ix-point_jx)**2+(point_iy-point_jy)**2)
            print(f"distance b/w point i = {i}, j={j}: {distance} units")
        

distance b/w point i = 0, j=1: 2.8284271247461903 units
distance b/w point i = 0, j=2: 5.656854249492381 units
distance b/w point i = 1, j=0: 2.8284271247461903 units
distance b/w point i = 1, j=2: 2.8284271247461903 units
distance b/w point i = 2, j=0: 5.656854249492381 units
distance b/w point i = 2, j=1: 2.8284271247461903 units
